In [1]:
import pickle

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalAveragePooling1D, Input, Activation, \
    BatchNormalization, MaxPooling1D, concatenate

from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf



def RedDNN(max_len=1600):
    now_pay_input_layer = Input((max_len, 257), name='now_pay')
    #embedding_layer = Embedding(257, embedding_size)(now_pay_input_layer)
    x = Conv1D(32, 3)(now_pay_input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(16)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    

    output_layer = Dense(1, activation='sigmoid')(x)
    return Model(inputs=[now_pay_input_layer], outputs=output_layer)

In [2]:
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def preprocessing_payload(payloads, max_len=1600):
    tmp = []
    for payload in payloads:
        byte_payload = []
        for i in range(0, len(payload), 2):
            byte_payload.append(int(payload[i:i + 2], 16))
        tmp.append(byte_payload)
    tmp = pad_sequences(tmp, maxlen=max_len, padding='post', truncating='post', value=256)

    return to_categorical(tmp)

In [3]:
def extract(data):
    temp_payload = []
    temp_y = []
    for value in data.values():
        for j in range(len(value)):
            temp_payload.append(value[j][0])
            temp_y.append(2 - value[j][1])
    return temp_payload, temp_y

In [4]:
def myGenerator():
    data = pd.read_pickle("E:/full_data/from_2018_8_to_2019_1.pkl")
    X, y = extract(data)
    y = np.asarray(y, dtype=np.float32)
    length = len(X)
    bs = length//1000
    while 1:
        for i in range(1001): # 1443 * 1000 + remain[437] = 1,443,437 -> # of training samples
            tmp_X = preprocessing_payload(X[i*bs:(i+1)*bs])
            yield tmp_X, y[i*bs:(i+1)*bs]

In [13]:
def evaluateGenerator():
    data = pd.read_pickle("E:/full_data/from_2019_2_to_2019_3.pkl")
    X, y = extract(data)
    y = np.asarray(y, dtype=np.float32)
    length = len(X)
    bs = length//1000
    while 1:
        for i in range(1001): # 723 * 1000 + remain[320] = 723,320 -> # of evaluate samples
            tmp_X = preprocessing_payload(X[i*bs:(i+1)*bs])
            yield tmp_X, y[i*bs:(i+1)*bs]

In [5]:
model = RedDNN()

In [7]:
ear = EarlyStopping(monitor='loss', patience=4)
ckpt = ModelCheckpoint(filepath=r"E:\full_data\now_one_hot_payload.h5", monitor='loss',
                       verbose=1, save_best_only=True)

In [8]:
model.fit_generator(myGenerator(), steps_per_epoch=1001, epochs = 16, verbose=1, callbacks=[ear, ckpt], validation_data=None, class_weight=None, workers=1, shuffle=False)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 1001 steps
Epoch 1/16
1000/1001 [============================>.] - ETA: 1s - loss: 0.5837 - acc: 0.7190 - precision: 0.7188 - recall: 0.5489
Epoch 00001: loss improved from inf to 0.58380, saving model to E:\full_data\now_no_embedding_only_payload.h5
1001/1001 [==============================] - 1163s 1s/step - loss: 0.5839 - acc: 0.7189 - precision: 0.7188 - recall: 0.5487
Epoch 2/16
1000/1001 [============================>.] - ETA: 1s - loss: 0.4838 - acc: 0.8057 - precision: 0.8503 - recall: 0.6549
Epoch 00002: loss improved from 0.58380 to 0.48393, saving model to E:\full_data\now_no_embedding_only_payload.h5
1001/1001 [==============================] - 1206s 1s/step - loss: 0.4841 - acc: 0.8056 - precision: 0.8503 - recall: 0.6546
Epoch 3/16
1000/1001 [============================>.] - ETA: 1s - loss: 0.4179 - acc: 0.8296 - precision: 0.8498 - recall: 0.7242
Epoch 00003: l

In [14]:
model.evaluate_generator(evaluateGenerator(), steps=1001)

Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']


[0.05492297212275754, 0.98312086, 0.9973903, 0.8262041]